<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/01_morphometrics_model_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Morphometrics Notebook 01

Script to extract following measurements from Detectron2 masks:

*   Major axis (e.g maximum diameter)
*   Minor axis (e.g minimum diameter)
*   Circularity
*   Ellipticity


Output: CSV file detailing above morphometrics for each nannofossil

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
detectron2  Detectron2_notebooks  Mask-RCNN  Morphometrics  output


In [ ]:
project_dir = "/content/drive/MyDrive/data/species_53"

## Load Detectron

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 6.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s 

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# Import Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog, detection_utils as utils, build_detection_train_loader
from detectron2.structures import BoxMode

In [ ]:
import yaml

## Load Configuration and Model

In [ ]:
# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"  # Path to your saved config file
cfg = get_cfg()

# Set default values for keys that might be missing in the config file
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Merge the configuration file with the default (debugged this way)
cfg.merge_from_file(config_yaml_path)
# Initialize the predictor with the loaded configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")

## Define Predictor and Metadata

In [ ]:
predictor = DefaultPredictor(cfg)

[06/13 15:19:01 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


In [ ]:
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])

## Morphometrics

In [ ]:
import csv
import cv2
from skimage.measure import regionprops, label
import numpy as np

In [ ]:
input_images_directory = "/content/drive/MyDrive/data/species_53/data/train"
output_csv_path = "/content/drive/MyDrive/data/species_53/morphometric_output/morphometrics_trials.csv"

In [ ]:
# Register datasets in the COCO format
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/data/species_53/data/train/coco_train.json", "/content/drive/MyDrive/data/species_53/data/train")

In [ ]:
thing_classes = ["t_orionatus"]
MetadataCatalog.get("my_dataset_train").thing_classes = thing_classes

In [ ]:
# Look at major axis

# import random
# # List all image filenames in the input directory
# image_filenames = [f for f in os.listdir(input_images_directory) if f.lower().endswith(('.jpeg', '.jpg'))]

# # Shuffle the list to get random images
# random.shuffle(image_filenames)

# # Select a subset of images (e.g., first 10 images)
# num_images_to_process = 10
# selected_image_filenames = image_filenames[:num_images_to_process]

# # Iterate over selected images
# for image_filename in selected_image_filenames:
#     image_path = os.path.join(input_images_directory, image_filename)
#     new_im = cv2.imread(image_path)

#     # Perform prediction on the images
#     outputs = predictor(new_im)
#     masks = outputs["instances"].pred_masks.to("cpu").numpy().astype(bool)

#     # Use skimage.measure.regionprops to calculate object parameters
#     labeled_mask = label(masks)
#     props = regionprops(labeled_mask)

#     for i, prop in enumerate(props):
#         object_number = i + 1

#         # Print out region properties for debugging
#         print(f"Image: {image_filename}")
#         print(f"Object {object_number}:")
#         print(f"  - Major Axis Length: {prop.major_axis_length}")


Image: PM_NF_5341_10_25.jpeg
Object 1:
  - Major Axis Length: 106.35722131354282
Image: PM_NF_5341_10_25.jpeg
Object 2:
  - Major Axis Length: 58.432155433928095
Image: PM_NF_5338_47_7.jpeg
Object 1:
  - Major Axis Length: 77.83203780186497
Image: PM_NF_5290_33_38.jpeg
Object 1:
  - Major Axis Length: 95.54655569453281
Image: PM_NF_5338_52_28.jpeg
Object 1:
  - Major Axis Length: 105.00077573471161
Image: PM_NF_5362_30_7.jpeg
Object 1:
  - Major Axis Length: 121.08334055693724
Image: PM_NF_5362_13_10.jpeg
Object 1:
  - Major Axis Length: 110.31208460048609
Image: PM_NF_5362_13_10.jpeg
Object 2:
  - Major Axis Length: 84.81164244009885
Image: PM_NF_5362_13_10.jpeg
Object 3:
  - Major Axis Length: 83.30797751641339


/content/drive/MyDrive/development/avellina/detectron2/detectron2/layers/wrappers.py:142: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  x = F.conv2d(


Image: PM_NF_5311_34_18.jpeg
Object 1:
  - Major Axis Length: 101.85167745104413
Image: PM_NF_5065_14_32.jpeg
Object 1:
  - Major Axis Length: 84.41568012646626
Image: PM_NF_5065_14_32.jpeg
Object 2:
  - Major Axis Length: 32.78884881514709
Image: PM_NF_5065_14_32.jpeg
Object 3:
  - Major Axis Length: 56.84933480842582
Image: PM_NF_5065_14_32.jpeg
Object 4:
  - Major Axis Length: 31.99658913003805
Image: PM_NF_5065_14_32.jpeg
Object 5:
  - Major Axis Length: 43.75687072885012
Image: PM_NF_5065_14_32.jpeg
Object 6:
  - Major Axis Length: 79.08602999623314
Image: PM_NF_5065_14_32.jpeg
Object 7:
  - Major Axis Length: 80.82940099617512
Image: PM_NF_5362_01_41.jpeg
Object 1:
  - Major Axis Length: 96.65827495050425
Image: PM_NF_5311_87_46.jpeg
Object 1:
  - Major Axis Length: 72.02868110437001
Image: PM_NF_5311_87_46.jpeg
Object 2:
  - Major Axis Length: 50.21230115499041


In [ ]:
# look at minor axis -- why some aren't working

# for i, prop in enumerate(props):
#     object_number = i + 1

#     # Print out region properties for debugging
#     print(f"Object {object_number}:")
#     print(f"  - Major Axis Length: {prop.major_axis_length}")

#     # Handle edge case where minor_axis_length might be very small or zero
#     try:
#         minor_axis_length = prop.minor_axis_length
#         if np.isnan(minor_axis_length) or minor_axis_length <= 0:
#             print(f"Warning: Object {object_number} has invalid minor_axis_length ({minor_axis_length}). Skipping.")
#             continue
#     except Exception as e:
#         print(f"Warning: Object {object_number} has error in minor_axis_length calculation: {e}. Skipping.")
#         continue

#     print(f"  - Minor Axis Length: {minor_axis_length}")


Object 1:
  - Major Axis Length: 72.02868110437001
  - Minor Axis Length: 3.769728732309794e-07
Object 2:
  - Major Axis Length: 50.21230115499041


In [ ]:
# Open CSV file for writing
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["File Name", "Class Name", "Object Number", "Max Diameter (um)", "Min Diameter (um)", "Circularity", "Ellipticity"])

    # Iterate over images in the input directory
    for image_filename in os.listdir(input_images_directory):
        if image_filename.lower().endswith(('.jpeg', '.jpg')):
            image_path = os.path.join(input_images_directory, image_filename)
            new_im = cv2.imread(image_path)

            # Perform prediction on the images
            outputs = predictor(new_im)
            masks = outputs["instances"].pred_masks.to("cpu").numpy().astype(bool)
            class_labels = outputs["instances"].pred_classes.to("cpu").numpy()

            # Use skimage.measure.regionprops to calculate object parameters
            labeled_mask = label(masks)

            # Check if the mask is empty
            if np.count_nonzero(labeled_mask) == 0:
                print(f"Warning: Image {image_filename} has an empty mask. Skipping.")
                continue

            props = regionprops(labeled_mask)

            scale = 0.0735  # Change this to the real scale later --> 340x340 pixels = 25x25 micrometers --> 25/340

            for i, prop in enumerate(props):
                object_number = i + 1

                # Print out region properties for debugging
                print(f"Image: {image_filename}, Object {object_number}:")
                print(f"  - Major Axis Length: {prop.major_axis_length}")

                # Handle edge case where minor_axis_length might be very small or zero -- debug solution
                try:
                    minor_axis_length = prop.minor_axis_length
                    if np.isnan(minor_axis_length) or minor_axis_length <= 0:
                        print(f"Warning: Object {object_number} in image {image_filename} has invalid minor_axis_length ({minor_axis_length}). Skipping.")
                        continue
                except Exception as e:
                    print(f"Warning: Object {object_number} in image {image_filename} has error in minor_axis_length calculation: {e}. Skipping.")
                    continue

                print(f"  - Minor Axis Length: {minor_axis_length}")

                max_diameter = prop.major_axis_length * scale
                min_diameter = prop.minor_axis_length * scale

                # Circularity and Ellipticity
                circularity = np.sqrt((min_diameter * max_diameter) / (max_diameter ** 2))
                ellipticity = max_diameter / min_diameter

                if i < len(class_labels):
                    class_label = class_labels[i]
                    class_name = metadata.thing_classes[class_label]
                else:
                    class_name = 'Unknown'

                # Save to CSV
                csvwriter.writerow([image_filename, class_name, object_number, max_diameter, min_diameter, circularity, ellipticity])

print("Morphometrics successfully saved to CSV file in morphometrics output folder.")  # Sanity check

Image: PM_NF_5408_45_37.jpeg, Object 1:
  - Major Axis Length: 53.76883221572623
  - Minor Axis Length: 3.769728732309794e-07
Image: PM_NF_5408_45_37.jpeg, Object 2:
  - Major Axis Length: 97.24719194517438
Image: PM_NF_5065_13_23.jpeg, Object 1:
  - Major Axis Length: 102.47754605372722
Image: PM_NF_5065_29_10.jpeg, Object 1:
  - Major Axis Length: 75.43205545985799
Image: PM_NF_5423_04_26.jpeg, Object 1:
  - Major Axis Length: 55.10007196860606
Image: PM_NF_5423_04_26.jpeg, Object 2:
  - Major Axis Length: 74.6702467182838
  - Minor Axis Length: 9.233911862867873e-07
Image: PM_NF_5423_04_26.jpeg, Object 3:
  - Major Axis Length: 130.30857844345007
Image: PM_NF_5423_04_26.jpeg, Object 4:
  - Major Axis Length: 31.412613836609587
Image: PM_NF_5065_20_23.jpeg, Object 1:
  - Major Axis Length: 90.76869674358655
Image: PM_NF_5408_22_6.jpeg, Object 1:
  - Major Axis Length: 94.41677487856474
  - Minor Axis Length: 7.539457464619588e-07
Image: PM_NF_5311_05_31.jpeg, Object 1:
  - Major Axis

/content/drive/MyDrive/development/avellina/detectron2/detectron2/layers/wrappers.py:142: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  x = F.conv2d(


Image: PM_NF_5065_05_21.jpeg, Object 1:
  - Major Axis Length: 87.22491762743441
  - Minor Axis Length: 5.331201499700045e-07
Image: PM_NF_5065_05_21.jpeg, Object 2:
  - Major Axis Length: 79.52667256189275
  - Minor Axis Length: 2.0407870663384364
Image: PM_NF_5065_05_21.jpeg, Object 3:
  - Major Axis Length: 44.15774337631651
Image: PM_NF_5065_05_21.jpeg, Object 4:
  - Major Axis Length: 102.55978270960331
  - Minor Axis Length: 7.539457464619588e-07
Image: PM_NF_5065_05_21.jpeg, Object 5:
  - Major Axis Length: 84.13110052843321
  - Minor Axis Length: 2.14142644793697
Image: PM_NF_5309_19_27.jpeg, Object 1:
  - Major Axis Length: 94.52937612200955
Image: PM_NF_5309_19_27.jpeg, Object 2:
  - Major Axis Length: 51.46210406154202
Image: PM_NF_5309_19_27.jpeg, Object 3:
  - Major Axis Length: 113.99469854823587
Image: PM_NF_5309_19_27.jpeg, Object 4:
  - Major Axis Length: 101.09468852383021
Image: PM_NF_5309_19_27.jpeg, Object 5:
  - Major Axis Length: 128.9986274889838
  - Minor Axis 

In [ ]:
#version with number of t orionatus

with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["file_name", "class_name", "number_of_t_orionatus", "object_number", "major_axis", "minor_axis", "circularity", "ellipticity"])

    # Iterate over images in the input directory
    for image_filename in os.listdir(input_images_directory):
        if image_filename.lower().endswith(('.jpeg', '.jpg')):
            image_path = os.path.join(input_images_directory, image_filename)
            new_im = cv2.imread(image_path)

            # Perform prediction on the images
            outputs = predictor(new_im)
            masks = outputs["instances"].pred_masks.to("cpu").numpy().astype(bool)
            class_labels = outputs["instances"].pred_classes.to("cpu").numpy()

            # Use skimage.measure.regionprops to calculate object parameters
            labeled_mask = label(masks)

            # Check if the mask is empty
            if np.count_nonzero(labeled_mask) == 0:
                print(f"Warning: Image {image_filename} has an empty mask. Skipping.")
                continue

            props = regionprops(labeled_mask)

            scale = 0.0735  # Change this to the real scale later

            # Count the number of t_orionatus objects
            number_of_t_orionatus = 0
            for i, prop in enumerate(props):
                if i < len(class_labels) and class_labels[i] == t_orionatus_label:
                    number_of_t_orionatus += 1

            # Initialize a list to store row data
            rows_to_write = []

            for i, prop in enumerate(props):
                object_number = i + 1

                # Print out region properties for debugging
                print(f"Image: {image_filename}, Object {object_number}:")
                print(f"  - Major Axis Length: {prop.major_axis_length}")

                # Handle edge case where minor_axis_length might be very small or zero
                try:
                    minor_axis_length = prop.minor_axis_length
                    if np.isnan(minor_axis_length) or minor_axis_length <= 0:
                        print(f"Warning: Object {object_number} in image {image_filename} has invalid minor_axis_length ({minor_axis_length}). Skipping.")
                        continue

                    print(f"  - Minor Axis Length: {minor_axis_length}")

                    max_diameter = prop.major_axis_length * scale
                    min_diameter = minor_axis_length * scale

                    # Circularity and Ellipticity
                    circularity = np.sqrt((min_diameter * max_diameter) / (max_diameter ** 2))
                    ellipticity = max_diameter / min_diameter

                    if i < len(class_labels):
                        class_label = class_labels[i]
                        class_name = metadata.thing_classes[class_label]
                    else:
                        class_name = 'Unknown'

                    # Append data to rows_to_write
                    rows_to_write.append([image_filename, class_name, number_of_t_orionatus, object_number, max_diameter, min_diameter, circularity, ellipticity])

                except Exception as e:
                    print(f"Warning: Object {object_number} in image {image_filename} has error in diameter calculation: {e}. Skipping.")
                    continue

            # Write all rows to CSV
            csvwriter.writerows(rows_to_write)

print("Morphometrics successfully saved to CSV file in morphometrics output folder.")  # Sanity check

Image: PM_NF_5408_45_37.jpeg, Object 1:
  - Major Axis Length: 53.76883221572623
  - Minor Axis Length: 3.769728732309794e-07
Image: PM_NF_5408_45_37.jpeg, Object 2:
  - Major Axis Length: 97.24719194517438
Image: PM_NF_5065_13_23.jpeg, Object 1:
  - Major Axis Length: 102.47754605372722
Image: PM_NF_5065_29_10.jpeg, Object 1:
  - Major Axis Length: 75.43205545985799
Image: PM_NF_5423_04_26.jpeg, Object 1:
  - Major Axis Length: 55.10007196860606
Image: PM_NF_5423_04_26.jpeg, Object 2:
  - Major Axis Length: 74.6702467182838
  - Minor Axis Length: 9.233911862867873e-07
Image: PM_NF_5423_04_26.jpeg, Object 3:
  - Major Axis Length: 130.30857844345007
Image: PM_NF_5423_04_26.jpeg, Object 4:
  - Major Axis Length: 31.412613836609587
Image: PM_NF_5065_20_23.jpeg, Object 1:
  - Major Axis Length: 90.76869674358655
Image: PM_NF_5408_22_6.jpeg, Object 1:
  - Major Axis Length: 94.41677487856474
  - Minor Axis Length: 7.539457464619588e-07
Image: PM_NF_5311_05_31.jpeg, Object 1:
  - Major Axis